__File: src_physics_write_data.ipynb__

__Author:   A. J. Tropiano (tropiano.4@osu.edu)__<br/>
__Date:     Apr 09, 2021__

Writes SRC physics data files for plotting purposes. Use these files with src_physics_fig.ipynb to generate figures relying only data files.

__Revision history:<br/>__
* Apr 13, 2021 - Finished the following functions: n_lambda_deuteron_decomposition, deuteron_partial_wave_contributions, srg_transformation_partial_wave_ratios, lda_pair_momentum_distributions, lda_single_nucleon_momentum_distributions, lda_deuteron_pair_momentum_distribution, lda_deuteron_single_nucleon_momentum_distribution, a2_scaling_factors, and deuteron_partial_wave_ratios.
* Apr 16, 2021 - Adding Fe56 to LDA data files.
* Apr 17, 2021 - Generating data for Gezerlis N2LO 1 and 1.2 fm potentials.
* Apr 18, 2021 - Adding option to save directly to Dropbox directory in addition to LENT repository.
* Apr 21, 2021 - Generating momentum distributions at $\lambda=1.5$ and $2$ fm$^{-1}$.
* Apr 28, 2021 - Generating data files for new momentum mesh with $k_{\rm max}=15$ fm$^{-1}$.
* May 07, 2021 - Added snmd_partial_wave_contributions function to test S-wave dominance.
* May 11, 2021 - Added factorization functions under Extras.
* May 27, 2021 - Updating data files after correcting integrations in pmd.py and snmd.py. Correcting integration mesh in $a_2$ function.
* Jun 4, 2021 - Evaluating factorization ratios for more $k_0$ values.
* Jun 11, 2021 - Updated to accommodate changes to pmd.py, snmd.py, and dmd.py (e.g., using interpolated functions instead of full momentum distribution calculations to speed things up).
* Jul 05, 2021 - Created src_physics_write_data_v1.ipynb with submission to PRC. All further updates are not included in the paper.
* Jul 14, 2021 - Adding Gogny and AV18 densities.
* Aug 17, 2021 - Adding absolute contacts function.

In [1]:
from os import chdir, getcwd
import numpy as np
import shutil
import time
# Scripts made by A.T.
from densities import load_density
from dmd import deuteron_momentum_distributions
from figures import figures_functions as ff
from misc.integration import gaussian_quadrature_mesh
import observables as ob
import operators as op
from pmd import pair_momentum_distributions
from potentials.vsrg_macos import vnn
from snmd import single_nucleon_momentum_distributions
from srg.srg_unitary_transformation import SRG_unitary_transformation

__Set-up__

In [2]:
# Current working directory
cwd = getcwd()

# Specify potential
kvnn = 6 # AV18
# kvnn = 222 # Gezerlis N2LO 1 fm
# kvnn = 224 # Gezerlis N2LO 1.2 fm

# Save data in the following directory
data_directory = 'figures/src_physics/data/kvnn_%d' % kvnn

# Option to also save files to Dropbox directory
dropbox_directory = '../../../Dropbox/SRG_projects/Operator_evolution/Figures/SRC_physics/Data/kvnn_%d' % kvnn
save_to_dropbox = False
# save_to_dropbox = True

# Specify which momentum mesh you want to use
# kmax, kmid, ntot = 10.0, 2.0, 120 # Mesh artifacts seen in last few points of 3S1-3D1 \delta U for AV18
kmax, kmid, ntot = 15.0, 3.0, 120

# SRG evolve to 1.35 fm^-1
lamb = 1.35 

# Channels to include in calculation of momentum distributions
channels = ('1S0', '3S1') # S-waves only
# channels = ('1S0', '3S1', '3P0', '1P1', '3P1') # Still having issue with '3P2' and '3D3'

# Nuclei to show
nuclei_sly4 = ( ('He4', 2, 2), ('C12', 6, 6), ('O16', 8, 8), ('Ca40', 20, 20), ('Ca48', 20, 28), ('Fe56', 26, 30),
                ('Pb208', 82, 126) )
nuclei_gogny = ( ('He4', 2, 2), ('Be9', 4, 5), ('C12', 6, 6), ('O16', 8, 8), ('Al27', 13, 14), ('Ca40', 20, 20),
                 ('Ca48', 20, 28), ('Fe56', 26, 30), ('Cu63', 29, 34), ('Au197', 79, 118), ('Pb208', 82, 126) )
nuclei_av18 = ( ('He4', 2, 2), ('He8', 2, 6), ('Be9', 4, 5) )

__Decomposition of $n^{\lambda}_d(q)$__

In [3]:
def n_lambda_deuteron_decomposition(kvnn, lamb, kmax=15.0, kmid=3.0, ntot=120, save_to_dropbox=False):
    """
    SRG-evolved deuteron pair momentum distribution as calculated by our expansion of U(k, k') in second
    quantization using deuteron wave function (not LDA). Gives each of the contributions from 1, \delta U,
    \delta U \delta U^\dagger.
    
    Parameters
    ----------
    kvnn : int
        This number specifies the potential.
    lamb : float
        SRG \lambda parameter [fm^-1].
    kmax : float, optional
        Maximum value in the momentum mesh [fm^-1].
    kmid : float, optional
        Mid-point value in the momentum mesh [fm^-1].
    ntot : int, optional
        Number of momentum points in mesh.
    save_to_dropbox : bool, optional
        Option to save file to Dropbox directory.
    
    """

    # Make file name
    file_name = 'n_lambda_deuteron_decomposition_kvnn_%d_lamb_%.2f_kmax_%.1f' % (kvnn, lamb, kmax)
    file_name = ff.replace_periods(file_name) + '.dat'

    # Load momentum
    q_array, _ = vnn.load_momentum(kvnn, '3S1', kmax, kmid, ntot)
    
    # Initialize deuteron momentum distributions class for given potential
    dmd = deuteron_momentum_distributions(kvnn, lamb, kmax, kmid, ntot, interp=False)

    # Open file and write header where we allocate roughly 18 centered spaces for each label
    f = open(data_directory + '/' + file_name, 'w')
    header = '#' + '{:^17s}{:^18s}{:^18s}{:^18s}{:^18s}'.format('q', 'total', '1 term', '\delta U term',
                                                                '\delta U^2 term')
    f.write(header + '\n')
    
    # Loop over momenta q
    for iq, q in enumerate(q_array):
        
        # Calculate each contribution to n_\lambda(q) including total
        total, term_1, term_deltaU, term_deltaU2 = dmd.n_lambda_exact(q, 'q_contributions')

        # Write to data file following the format from the header
        line = '{:^18.6f}{:^18.6e}{:^18.6e}{:^18.6e}{:^18.6e}'.format(q, total, term_1, term_deltaU,
                                                                      term_deltaU2)
        f.write('\n' + line)

    # Close file
    f.close()
    
    # Save to Dropbox?
    if save_to_dropbox:
        _ = shutil.copy(data_directory + '/' + file_name, dropbox_directory)

In [4]:
# # Contributions to deuteron momentum distribution in terms of 1, \delta U, and \delta U^2

# t0 = time.time() # Start time
# n_lambda_deuteron_decomposition(kvnn, lamb, kmax, kmid, ntot, save_to_dropbox)
# t1 = time.time() # Stop time
# mins = round( (t1 - t0) / 60.0, 5) # Minutes elapsed
# print('%.5f minutes elasped' % mins)

__Deuteron partial wave contributions__

In [5]:
def deuteron_partial_wave_contributions(kvnn, lamb, kmax=15.0, kmid=3.0, ntot=120, save_to_dropbox=False):
    """
    Contributions of each partial wave coupled-channel to |\psi_d(q)|^2 comparing unevolved and SRG-evolved
    for various momenta q [fm^-1].
    
    Parameters
    ----------
    kvnn : int
        This number specifies the potential.
    lamb : float
        SRG \lambda parameter [fm^-1].
    kmax : float, optional
        Maximum value in the momentum mesh [fm^-1].
    kmid : float, optional
        Mid-point value in the momentum mesh [fm^-1].
    ntot : int, optional
        Number of momentum points in mesh.
    save_to_dropbox : bool, optional
        Option to save file to Dropbox directory.
    
    """

    # Make file name
    file_name = 'deuteron_partial_wave_contributions_kvnn_%d_lamb_%.2f_kmax_%.1f' % (kvnn, lamb, kmax)
    file_name = ff.replace_periods(file_name) + '.dat'

    # Channel is 3S1-3D1 for deuteron
    channel = '3S1'
    
    # Load momentum [fm^-1]
    q_array, q_weights = vnn.load_momentum(kvnn, channel, kmax, kmid, ntot)
    
    # Load initial and evolved Hamiltonians [MeV]
    H_initial = vnn.load_hamiltonian(kvnn, channel, kmax, kmid, ntot)
    H_evolved = vnn.load_hamiltonian(kvnn, channel, kmax, kmid, ntot, method='srg', generator='Wegner',
                                     lamb=lamb)
    
    # Calculate SRG transformation [unitless]
    U_matrix = SRG_unitary_transformation(H_initial, H_evolved)

    # Load initial and evolved wave functions [unitless]
    psi_initial = ob.wave_function(H_initial)
    psi_evolved = ob.wave_function(H_initial, U=U_matrix)

    # Open file and write header where we allocate roughly 18 centered spaces for each label
    f = open(data_directory + '/' + file_name, 'w')
    header = '#' + '{:^17s}{:^18s}{:^18s}{:^18s}{:^18s}{:^18s}{:^18s}'.format('q', '3S1-3S1 init.',
             '3S1-3D1 init.', '3D1-3D1 init.', '3S1-3S1 SRG', '3S1-3D1 SRG', '3D1-3D1 SRG')
    f.write(header + '\n')

    # Loop over momenta q
    for iq, q in enumerate(q_array):
        
        # Initial and evolved momentum projection operators
        operator_initial = op.momentum_projection_operator(q, q_array, q_weights, channel, smeared=False)
        operator_evolved = op.momentum_projection_operator(q, q_array, q_weights, channel, U_matrix,
                                                           smeared=False)
    
        # Each contribution to |\psi_d(q)|^2 for initial with total
        psi2_3S1_3S1_initial = abs( psi_initial.T[:ntot] @ operator_initial[:ntot, :ntot] @ psi_initial[:ntot] )
        psi2_3S1_3D1_initial = abs( psi_initial.T[:ntot] @ operator_initial[:ntot, ntot:] @ psi_initial[ntot:] )
        psi2_3D1_3S1_initial = abs( psi_initial.T[ntot:] @ operator_initial[ntot:, :ntot] @ psi_initial[:ntot] )
        psi2_3D1_3D1_initial = abs( psi_initial.T[ntot:] @ operator_initial[ntot:, ntot:] @ psi_initial[ntot:] )
        psi2_initial = psi2_3S1_3S1_initial + psi2_3S1_3D1_initial + psi2_3D1_3S1_initial + psi2_3D1_3D1_initial

        # Each contribution to |\psi_d(q)|^2 for evolved with total
        psi2_3S1_3S1_evolved = abs( psi_evolved.T[:ntot] @ operator_evolved[:ntot, :ntot] @ psi_evolved[:ntot] )
        psi2_3S1_3D1_evolved = abs( psi_evolved.T[:ntot] @ operator_evolved[:ntot, ntot:] @ psi_evolved[ntot:] )
        psi2_3D1_3S1_evolved = abs( psi_evolved.T[ntot:] @ operator_evolved[ntot:, :ntot] @ psi_evolved[:ntot] )
        psi2_3D1_3D1_evolved = abs( psi_evolved.T[ntot:] @ operator_evolved[ntot:, ntot:] @ psi_evolved[ntot:] )
        psi2_evolved = psi2_3S1_3S1_evolved + psi2_3S1_3D1_evolved + psi2_3D1_3S1_evolved + psi2_3D1_3D1_evolved
        
        # Calculate the percentage of each contribution
        cont_3S1_3S1_initial = psi2_3S1_3S1_initial / psi2_initial * 100
        cont_3S1_3D1_initial = psi2_3S1_3D1_initial / psi2_initial * 100 * 2 # Factor of 2 for 3D1-3S1
        cont_3D1_3D1_initial = psi2_3D1_3D1_initial / psi2_initial * 100
        cont_3S1_3S1_evolved = psi2_3S1_3S1_evolved / psi2_evolved * 100
        cont_3S1_3D1_evolved = psi2_3S1_3D1_evolved / psi2_evolved * 100 * 2 # Factor of 2 for 3D1-3S1
        cont_3D1_3D1_evolved = psi2_3D1_3D1_evolved / psi2_evolved * 100
        
        # Write to data file following the format from the header
        line = '{:^18.6f}{:^18.6f}{:^18.6f}{:^18.6f}{:^18.6f}{:^18.6f}{:^18.6f}'.format(q, cont_3S1_3S1_initial,
                cont_3S1_3D1_initial, cont_3D1_3D1_initial, cont_3S1_3S1_evolved, cont_3S1_3D1_evolved,
                cont_3D1_3D1_evolved)
        f.write('\n' + line)

    # Close file
    f.close()
    
    # Save to Dropbox?
    if save_to_dropbox:
        _ = shutil.copy(data_directory + '/' + file_name, dropbox_directory)

In [6]:
# # Deuteron partial wave contributions |\psi_d(q)|^2

# t0 = time.time() # Start time
# deuteron_partial_wave_contributions(kvnn, lamb, kmax, kmid, ntot, save_to_dropbox)
# t1 = time.time() # Stop time
# mins = round( (t1 - t0) / 60.0, 5) # Minutes elapsed
# print('%.5f minutes elasped' % mins)

__($^{3}S_1$) / $^{1}S_0$ SRG transformation ratio__

In [7]:
def srg_transformation_partial_wave_ratios(k_0, kvnn, lamb, kmax=15.0, kmid=3.0, ntot=120,
                                           save_to_dropbox=False):
    """
    Ratio of |U(k_0, q)_{3S1}|^2 / |U(k_0, q)_{1S0}|^2 where k_0 < \lambda.
    
    Parameters
    ----------
    k_0 : float
        k_0 value in the ratio U(k_0, q)_{3S1} / U(k_0, q)_{1S0} [fm^-1].
    kvnn : int
        This number specifies the potential.
    lamb : float
        SRG \lambda parameter [fm^-1].
    kmax : float, optional
        Maximum value in the momentum mesh [fm^-1].
    kmid : float, optional
        Mid-point value in the momentum mesh [fm^-1].
    ntot : int, optional
        Number of momentum points in mesh.
    save_to_dropbox : bool, optional
        Option to save file to Dropbox directory.

    """

    # Make file name
    file_name = 'srg_transformation_partial_wave_ratios_k_0_%.1f_kvnn_%d_lamb_%.2f_kmax_%.1f' % (k_0, kvnn,
                                                                                                 lamb, kmax)
    file_name = ff.replace_periods(file_name) + '.dat'

    # Load momentum and weights (channel argument doesn't matter here)
    k_array, k_weights = vnn.load_momentum(kvnn, '1S0', kmax, kmid, ntot)
    
    # Divide out integration factors with factor_array
    factor_array = np.sqrt( (2*k_weights) / np.pi ) * k_array
    row, col = np.meshgrid(factor_array, factor_array)
    # Same thing but for coupled-channel
    factor_array_cc = np.concatenate( (factor_array, factor_array) )
    row_cc, col_cc = np.meshgrid(factor_array_cc, factor_array_cc)
    
    # Load initial and evolved Hamiltonians [MeV], and SRG transformations
    # 1S0
    H_initial_1s0 = vnn.load_hamiltonian(kvnn, '1S0', kmax, kmid, ntot)
    H_evolved_1s0 = vnn.load_hamiltonian(kvnn, '1S0', kmax, kmid, ntot, method='srg', lamb=lamb)
    # 3S1
    H_initial_3s1 = vnn.load_hamiltonian(kvnn, '3S1', kmax, kmid, ntot,)
    H_evolved_3s1 = vnn.load_hamiltonian(kvnn, '3S1', kmax, kmid, ntot, method='srg', lamb=lamb)
    
    # Calculate SRG transformation with eigenvectors of initial and evolved Hamiltonians dividing out 
    # integration factors
    U_matrix_1s0 = SRG_unitary_transformation(H_initial_1s0, H_evolved_1s0) / row / col
    U_matrix_3s1 = SRG_unitary_transformation(H_initial_3s1, H_evolved_3s1) / row_cc / col_cc
    
    # Index of k_0 in k_array
    k_0_index = op.find_q_index(k_0, k_array)
        
    # Calculate |U(k_0, q)_{3S1}|^2
    numerator_array = U_matrix_3s1[k_0_index, :ntot] * U_matrix_3s1.T[:ntot, k_0_index] + \
                      U_matrix_3s1[k_0_index, ntot:] * U_matrix_3s1.T[ntot:, k_0_index]
        
    # Calculate |U(k, q_i)_{1S0}|^2
    denominator_array = U_matrix_1s0[k_0_index, :ntot] * U_matrix_1s0.T[:ntot, k_0_index]
        
    # Take ratio of 3S1 / 1S0
    ratio_array = numerator_array / denominator_array
    
    # Open file and write header where we allocate roughly 18 centered spaces for each label
    f = open(data_directory + '/' + file_name, 'w')
    header = '#' + '{:^17s}{:^18s}'.format('k', 'ratio')
    f.write(header + '\n')
    
    # Loop over momenta k
    for ik, k in enumerate(k_array):

        # Write to data file following the format from the header
        line = '{:^18.6f}{:^18.6e}'.format(k, ratio_array[ik])
        f.write('\n' + line)

    # Close file
    f.close()
    
    # Save to Dropbox?
    if save_to_dropbox:
        _ = shutil.copy(data_directory + '/' + file_name, dropbox_directory)

In [8]:
# # U_{3S1} / U_{1S0} ratios with k_0=0.1 fm^-1

# k_0 = 0.1

# t0 = time.time() # Start time
# srg_transformation_partial_wave_ratios(k_0, kvnn, lamb, kmax, kmid, ntot, save_to_dropbox)
# t1 = time.time() # Stop time
# mins = round( (t1 - t0) / 60.0, 5) # Minutes elapsed
# print('%.5f minutes elasped' % mins)

__Pair momentum distributions $n_{\lambda}^A(q,Q=0)$__

In [9]:
def lda_pair_momentum_distributions(nucleus, channels, kvnn, lamb, kmax=15.0, kmid=3.0, ntot=120, edf='SLY4',
                                    save_to_dropbox=False):
    """
    SRG-evolved pair momentum distributions n_\lambda^A(q,Q=0) using HF+LDA.
    
    Parameters
    ----------
    nucleus : tuple
        Details for various nuclei formatted as a tuple: (name (str), Z (int), N (int)) (e.g., ('O16', 8, 8)).
    channels : tuple
        Partial wave channels to include in the calculation (e.g., ('1S0', '3S1')).
    kvnn : int
        This number specifies the potential.
    lamb : float
        SRG \lambda parameter [fm^-1].
    kmax : float, optional
        Maximum value in the momentum mesh [fm^-1].
    kmid : float, optional
        Mid-point value in the momentum mesh [fm^-1].
    ntot : int, optional
        Number of momentum points in mesh.
    edf : str, optional
        Name of EDF (e.g., 'SLY4').
    save_to_dropbox : bool, optional
        Option to save file to Dropbox directory.

    """

    # Name of nucleus, proton number, neutron number, and mass number
    nucleus_name = nucleus[0]
    Z = nucleus[1]
    N = nucleus[2]
    A = N + Z
    
    # Make file name
    file_name = 'lda_pmd_%s_channels' % nucleus_name
    # Add each channel to file name
    for channel in channels:
        file_name += '_%s' % channel
    file_name += '_kvnn_%d_lamb_%.2f_kmax_%.1f' % (kvnn, lamb, kmax)
    file_name = ff.replace_periods(file_name) + '.dat'

    # Load momentum (channel argument doesn't matter here)
    q_array, _ = vnn.load_momentum(kvnn, '3S1', kmax, kmid, ntot)
    
    # Initialize pair momentum distributions class for given potential
    pmd = pair_momentum_distributions(kvnn, channels, lamb, kmax, kmid, ntot, interp=False)
    
    # Load R values and nucleonic densities (the R_array's are the same)
    R_array, rho_p_array = load_density(nucleus_name, 'proton', Z, N, edf)
    R_array, rho_n_array = load_density(nucleus_name, 'neutron', Z, N, edf)
    dR = R_array[2] -  R_array[1] # Step-size

    # Calculate pair momentum distributions at Q = 0
    # These give each contribution total, 1, \delta U, and \delta U^2, so take total only!
    n_pp_array = pmd.n_Q0(q_array, R_array, dR, rho_p_array)
    n_pn_array = pmd.n_Q0(q_array, R_array, dR, rho_p_array, rho_n_array)
    # np is the same as pn (make sure you still account for np in figures, that is,
    # you will need to multiply pn by 2)
    n_nn_array = pmd.n_Q0(q_array, R_array, dR, rho_n_array)
    
    # Open file and write header where we allocate roughly 18 centered spaces for each label
    f = open(data_directory + '/%s/' % edf + file_name, 'w')
    header = '#' + '{:^17s}{:^18s}{:^18s}{:^18s}'.format('q', 'pp', 'pn', 'nn')
    f.write(header + '\n')
    
    # Loop over momenta q
    for iq, q in enumerate(q_array):

        # Write to data file following the format from the header
        line = '{:^18.6f}{:^18.6e}{:^18.6e}{:^18.6e}'.format(q, n_pp_array[iq], n_pn_array[iq], n_nn_array[iq])
        f.write('\n' + line)

    # Close file
    f.close()
    
    # Save to Dropbox?
    if save_to_dropbox:
        _ = shutil.copy(data_directory + '/' + file_name, dropbox_directory)

In [10]:
# # Pair momentum distributions using SLy4

# t0 = time.time() # Start time
# # Loop over nuclei
# for inucleus in nuclei_sly4:
#     lda_pair_momentum_distributions(inucleus, channels, kvnn, lamb, kmax, kmid, ntot, 'SLY4', save_to_dropbox)
# t1 = time.time() # Stop time
# mins = round( (t1 - t0) / 60.0, 5) # Minutes elapsed
# print('%.5f minutes elasped' % mins)

In [11]:
# # Pair momentum distributions using SLy4 with \lambda=1.5, 2, and 6 fm^-1

# lambdas = (1.5, 2.0, 6.0)

# t0 = time.time() # Start time
# # Loop over nuclei
# for inucleus in nuclei_sly4:
#     for ilamb in lambdas:
#         lda_pair_momentum_distributions(inucleus, channels, kvnn, ilamb, kmax, kmid, ntot, 'SLY4', save_to_dropbox)
# t1 = time.time() # Stop time
# mins = round( (t1 - t0) / 60.0, 5) # Minutes elapsed
# print('%.5f minutes elasped' % mins)

In [12]:
# # Pair momentum distributions using Gogny

# t0 = time.time() # Start time
# # Loop over nuclei
# for inucleus in nuclei_gogny:
#     lda_pair_momentum_distributions(inucleus, channels, kvnn, lamb, kmax, kmid, ntot, 'Gogny', save_to_dropbox)
# t1 = time.time() # Stop time
# mins = round( (t1 - t0) / 60.0, 5) # Minutes elapsed
# print('%.5f minutes elasped' % mins)

In [13]:
# # Pair momentum distributions using AV18

# t0 = time.time() # Start time
# # Loop over nuclei
# for inucleus in nuclei_av18:
#     lda_pair_momentum_distributions(inucleus, channels, kvnn, lamb, kmax, kmid, ntot, 'AV18', save_to_dropbox)
# t1 = time.time() # Stop time
# mins = round( (t1 - t0) / 60.0, 5) # Minutes elapsed
# print('%.5f minutes elasped' % mins)

__Single-nucleon momentum distributions $n_{\lambda}^A(q)$__

In [14]:
def lda_single_nucleon_momentum_distributions(nucleus, channels, kvnn, lamb, kmax=15.0, kmid=3.0, ntot=120, edf='SLY4',
                                              save_to_dropbox=False):
    """
    SRG-evolved single-nucleon momentum distributions n_\lambda^A(q) using HF+LDA.
    
    Parameters
    ----------
    nucleus : tuple
        Details for various nuclei formatted as a tuple: (name (str), Z (int), N (int)) (e.g., ('O16', 8, 8)).
    channels : tuple
        Partial wave channels to include in the calculation (e.g., ('1S0', '3S1')).
    kvnn : int
        This number specifies the potential.
    lamb : float
        SRG \lambda parameter [fm^-1].
    kmax : float, optional
        Maximum value in the momentum mesh [fm^-1].
    kmid : float, optional
        Mid-point value in the momentum mesh [fm^-1].
    ntot : int, optional
        Number of momentum points in mesh.
    edf : str, optional
        Name of EDF (e.g., 'SLY4').
    save_to_dropbox : bool, optional
        Option to save file to Dropbox directory.
        
    """

    # Name of nucleus, proton number, neutron number, and mass number
    nucleus_name = nucleus[0]
    Z = nucleus[1]
    N = nucleus[2]
    A = N + Z
    
    # Make file name
    file_name = 'lda_snmd_%s_channels' % nucleus_name
    # Add each channel to file name
    for channel in channels:
        file_name += '_%s' % channel
    file_name += '_kvnn_%d_lamb_%.2f_kmax_%.1f' % (kvnn, lamb, kmax)
    file_name = ff.replace_periods(file_name) + '.dat'

    # Load momentum (channel argument doesn't matter here)
    q_array, _ = vnn.load_momentum(kvnn, '3S1', kmax, kmid, ntot)
    
    # Initialize single-nucleon momentum distributions class for given potential
    snmd = single_nucleon_momentum_distributions(kvnn, channels, lamb, kmax, kmid, ntot, interp=True)

    # Get interpolated functions of proton and neutron momentum distributions
    # Ignore the 1, \delta U, and \delta U^2 isolated contributions (take total only)
    n_p_func, _, _, _ = snmd.n_lambda_interp(nucleus_name, 'proton', Z, N, edf)
    n_n_func, _, _, _ = snmd.n_lambda_interp(nucleus_name, 'neutron', Z, N, edf)
    
    # Calculate each distribution for momenta q
    n_p_array = n_p_func(q_array)
    n_n_array = n_n_func(q_array)
    
    # Open file and write header where we allocate roughly 18 centered spaces for each label
    f = open(data_directory + '/%s/' % edf + file_name, 'w')
    header = '#' + '{:^17s}{:^18s}{:^18s}'.format('q', 'proton', 'neutron')
    f.write(header + '\n')
    
    # Loop over momenta q
    for iq, q in enumerate(q_array):

        # Write to data file following the format from the header
        line = '{:^18.6f}{:^18.6e}{:^18.6e}'.format(q, n_p_array[iq], n_n_array[iq])
        f.write('\n' + line)

    # Close file
    f.close()
    
    # Save to Dropbox?
    if save_to_dropbox:
        _ = shutil.copy(data_directory + '/' + file_name, dropbox_directory)

In [15]:
# # Single-nucleon momentum distributions using SLy4

# t0 = time.time() # Start time
# # Loop over nuclei
# for inucleus in nuclei_sly4:
#     lda_single_nucleon_momentum_distributions(inucleus, channels, kvnn, lamb, kmax, kmid, ntot, 'SLY4',
#                                               save_to_dropbox)
# t1 = time.time() # Stop time
# mins = round( (t1 - t0) / 60.0, 5) # Minutes elapsed
# print('%.5f minutes elasped' % mins)

In [16]:
# # Single-nucleon momentum distributions using SLy4 with \lambda=1.5, 2, and 6 fm^-1

# lambdas = (1.5, 2.0, 6.0)

# t0 = time.time() # Start time
# # Loop over nuclei
# for inucleus in nuclei_sly4:
#     for ilamb in lambdas:
#         lda_single_nucleon_momentum_distributions(inucleus, channels, kvnn, ilamb, kmax, kmid, ntot, 'SLY4',
#                                                   save_to_dropbox)
# t1 = time.time() # Stop time
# mins = round( (t1 - t0) / 60.0, 5) # Minutes elapsed
# print('%.5f minutes elasped' % mins)

In [17]:
# # Single-nucleon momentum distributions using Gogny

# t0 = time.time() # Start time
# # Loop over nuclei
# for inucleus in nuclei_gogny:
#     lda_single_nucleon_momentum_distributions(inucleus, channels, kvnn, lamb, kmax, kmid, ntot, 'Gogny',
#                                               save_to_dropbox)
# t1 = time.time() # Stop time
# mins = round( (t1 - t0) / 60.0, 5) # Minutes elapsed
# print('%.5f minutes elasped' % mins)

In [18]:
# # Single-nucleon momentum distributions using AV18

# t0 = time.time() # Start time
# # Loop over nuclei
# for inucleus in nuclei_av18:
#     lda_single_nucleon_momentum_distributions(inucleus, channels, kvnn, lamb, kmax, kmid, ntot, 'AV18',
#                                               save_to_dropbox)
# t1 = time.time() # Stop time
# mins = round( (t1 - t0) / 60.0, 5) # Minutes elapsed
# print('%.5f minutes elasped' % mins)

__Partial wave contributions to $n_{\lambda}^A(q)$__

In [19]:
def snmd_partial_wave_contributions(nuclei, channels, kvnn, lamb, kmax=15.0, kmid=3.0, ntot=120, edf='SLY4',
                                    save_to_dropbox=False):
    """
    Fraction of partial wave contributions to total normalization of the high-q tail in nuclear-averaged
    and SRG-evolved single-nucleon momentum distributions n_\lambda^A(q). This is done by computing
        \int_2^\infty dq q^2 < n_A^p(q) >_X / \int_2^\infty dq q^2 < n_A^p(q) >,
    where X is a single partial wave channel. (Overall factor in front of integrals do not matter here.)
    
    Parameters
    ----------
    nuclei : tuple
        Tuple that contains the details for various nuclei formatted as another tuple:
        (name (str), Z (int), N (int)) (e.g., nuclei[i] = ('O16', 8, 8)).
    channels : tuple
        Partial wave channels to compute (e.g., ('1S0', '3S1', '3P0', '1P1', '3P1')).
    kvnn : int
        This number specifies the potential.
    lamb : float
        SRG \lambda parameter [fm^-1].
    kmax : float, optional
        Maximum value in the momentum mesh [fm^-1].
    kmid : float, optional
        Mid-point value in the momentum mesh [fm^-1].
    ntot : int, optional
        Number of momentum points in mesh.
    edf : str, optional
        Name of EDF (e.g., 'SLY4').
    save_to_dropbox : bool, optional
        Option to save file to Dropbox directory.
        
    """
    
    # Normalization factor (for printing purposes - divides out in ratio)
    factor = 4*np.pi / (2*np.pi)**3
    
    # Make file name
    file_name = 'snmd_partial_wave_contributions_nuclei'
    # Add each nucleus to file name
    for nucleus in nuclei:
        file_name += '_%s' % nucleus[0]
    file_name += '_channels'
    # Add each channel to file name
    for channel in channels:
        file_name += '_%s' % channel
    file_name += '_kvnn_%d_lamb_%.2f_kmax_%.1f' % (kvnn, lamb, kmax)
    file_name = ff.replace_periods(file_name) + '.dat'

    # Create momentum mesh for integration from 2-\infty fm^-1
    q_min = 2.0
    q_max = kmax # Corresponds to \infinity
    ntot_q = 80
    q_array, q_weights = gaussian_quadrature_mesh(q_max, ntot_q, xmin=q_min)
    
    # Calculate total proton momentum distributions first
    
    # Initialize single-nucleon momentum distributions class for given potential and all partial waves
    snmd_all = single_nucleon_momentum_distributions(kvnn, channels, lamb, kmax, kmid, ntot, interp=True)
    
    # Store total proton momentum distribution for each nucleus in dictionary
    d = {}
    for nucleus in nuclei:
        
        # Name of nucleus, proton number, neutron number, and mass number
        nucleus_name = nucleus[0]
        Z = nucleus[1]
        N = nucleus[2]
        A = N + Z
        
        # Make nucleus name the key
        d[nucleus_name] = {}

        # Load R values and nucleonic densities (the R_array's are the same)
        R_array, rho_p_array = load_density(nucleus_name, 'proton', Z, N, edf)
        R_array, rho_n_array = load_density(nucleus_name, 'neutron', Z, N, edf)
        dR = R_array[2] - R_array[1] # Step-size
        
        # Get interpolated functions of the proton momentum distributions for all partial waves
        # Ignore the 1, \delta U, and \delta U^2 isolated contributions (take total only)
        n_p_func_all, _, _, _ = snmd_all.n_lambda_interp(nucleus_name, 'proton', Z, N)
    
        # Calculate nuclear-averaged proton momentum distribution for all partial waves
        d[nucleus_name]['total'] = n_p_func_all(q_array)
        
        # Loop over each partial wave channel and calculate contribution
        for channel in channels:
            
            # This makes it so you can loop over the single channel (otherwise snmd.py wouldn't work)
            channel_tuple = (channel,)
            
            # Initialize pair momentum distributions class for given potential and one partial wave
            snmd_partial = single_nucleon_momentum_distributions(kvnn, channel_tuple, lamb, kmax, kmid, ntot,
                                                                 interp=False)
            
            # Calculate the proton momentum distributions for single partial wave
            d[nucleus_name][channel] = snmd_partial.n_total(q_array, R_array, dR, rho_p_array, rho_n_array)

    # Open file and write header where we allocate roughly 18 centered spaces for each label
    f = open(data_directory + '/%s/' % edf + file_name, 'w')
    
    # Header should show nuclei
    header = '{:^18s}'.format('   ')
    
    # Loop over nuclei
    for nucleus in nuclei:
        
        # Add each nucleus as a column label
        header += '{:^18s}'.format(nucleus[0])
    
    # Write header
    f.write(header + '\n')
        
    # Loop over partial wave channels
    for channel in channels:
        
        # Add each channel as a row label
        line = '{:^18s}'.format(channel)
        
        # Loop over nuclei to write to each column
        for nucleus in nuclei:
            
            # Compute partial wave contribution to tail normalization (without 4\pi/(2\pi)^3)
            numerator = np.sum( q_weights * q_array**2 * d[ nucleus[0] ][channel] )
            
            # Compute total tail normalization (without 4\pi/(2\pi)^3)
            denominator = np.sum( q_weights * q_array**2 * d[ nucleus[0] ]['total'] )
    
            # Add fraction to row
            line += '{:^18e}'.format(numerator/denominator)
            
        # Write row
        f.write(line + '\n')
        
    # Close file
    f.close()
    
    # Save to Dropbox?
    if save_to_dropbox:
        _ = shutil.copy(data_directory + '/' + file_name, dropbox_directory)

In [20]:
# # Partial wave contributions to tail of single-nucleon momentum distribution using SLy4

# t0 = time.time() # Start time
# snmd_partial_wave_contributions(nuclei_sly4, ('1S0', '3S1', '3P0', '1P1', '3P1'), kvnn, lamb, kmax, kmid, ntot,
#                                 'SLY4', save_to_dropbox)
# t1 = time.time() # Stop time
# mins = round( (t1 - t0) / 60.0, 5) # Minutes elapsed
# print('%.5f minutes elasped' % mins)

__Deuteron single-nucleon momentum distribution $n_{\lambda}^d(q)$ under HF+LDA__

In [21]:
def lda_deuteron_single_nucleon_momentum_distribution(kvnn, lamb, kmax=15.0, kmid=3.0, ntot=120,
                                                      save_to_dropbox=False):
    """
    SRG-evolved single-nucleon momentum distributions n_\lambda^A(q) using HF+LDA.
    
    Parameters
    ----------
    kvnn : int
        This number specifies the potential.
    lamb : float
        SRG \lambda parameter [fm^-1].
    kmax : float, optional
        Maximum value in the momentum mesh [fm^-1].
    kmid : float, optional
        Mid-point value in the momentum mesh [fm^-1].
    ntot : int, optional
        Number of momentum points in mesh.
    save_to_dropbox : bool, optional
        Option to save file to Dropbox directory.
        
    Notes
    -----
    Looks like there is a bug for last q-value (always zero).

    """

    # Make file name
    file_name = 'lda_snmd_H2_channels_3S1_kvnn_%d_lamb_%.2f_kmax_%.1f' % (kvnn, lamb, kmax)
    file_name = ff.replace_periods(file_name) + '.dat'

    # Load momentum (channel argument doesn't matter here)
    q_array, _ = vnn.load_momentum(kvnn, '3S1', kmax, kmid, ntot)

    # Initialize deuteron momentum distributions class for given potential
    dmd = deuteron_momentum_distributions(kvnn, lamb, kmax, kmid, ntot, interp=True)
    
    # Get interpolated functions of deuteron momentum distribution
    # Ignore the 1, \delta U, and \delta U^2 isolated contributions (take total only)
    n_d_func, _, _, _ = dmd.n_lambda_interp()
    
    # Calculate deuteron momentum distribution
    n_d_array = n_d_func(q_array)

    # Open file and write header where we allocate roughly 18 centered spaces for each label
    f = open(data_directory + '/' + file_name, 'w')
    header = '#' + '{:^17s}{:^18s}'.format('q', 'single-nucleon')
    f.write(header + '\n')
    
    # Loop over momenta q
    for iq, q in enumerate(q_array):

        # Write to data file following the format from the header
        line = '{:^18.6f}{:^18.6e}'.format(q, n_d_array[iq])
        f.write('\n' + line)

    # Close file
    f.close()
    
    # Save to Dropbox?
    if save_to_dropbox:
        _ = shutil.copy(data_directory + '/' + file_name, dropbox_directory)

In [22]:
# # Deuteron single-nucleon momentum distribution

# t0 = time.time() # Start time
# lda_deuteron_single_nucleon_momentum_distribution(kvnn, lamb, kmax, kmid, ntot, save_to_dropbox)
# t1 = time.time() # Stop time
# mins = round( (t1 - t0) / 60.0, 5) # Minutes elapsed
# print('%.5f minutes elasped' % mins)

In [23]:
# # Deuteron single-nucleon momentum distribution with \lambda=1.5, 2, and 6 fm^-1

# lambdas = (1.5, 2.0, 6.0)

# t0 = time.time() # Start time
# for ilamb in lambdas:
#     lda_deuteron_single_nucleon_momentum_distribution(kvnn, ilamb, kmax, kmid, ntot, save_to_dropbox)
# t1 = time.time() # Stop time
# mins = round( (t1 - t0) / 60.0, 5) # Minutes elapsed
# print('%.5f minutes elasped' % mins)

__$a_2$ scaling factors__

In [24]:
def a2_scaling_factors(nuclei, channels, kvnn, lamb, integration_limits=(2.0,), kmax=15.0, kmid=3.0,
                       ntot=120, edf='SLY4', save_to_dropbox=False):
    """
    a_2 scaling factors evaluated using HF+LDA, SRG-evolved single-nucleon momentum distributions.
    These factors are defined by
        ( \int_{q_start}^{q_stop} dq P^A(q) ) / ( \int_{q_start}^{q_stop} dq P^d(q) ),
    where P^A(q) is the single-nucleon probability distribution defined by P^A(q) = q^2 n^A(q) / A.
    The integration limits are set to 2-\infty fm^-1 by default.
    
    Parameters
    ----------
    nuclei : tuple
        Tuple that contains the details for various nuclei formatted as another tuple:
        (name (str), Z (int), N (int)) (e.g., nuclei[i] = ('O16', 8, 8)).
    channels : tuple
        Partial wave channels to include in the calculation (e.g., ('1S0', '3S1')).
    kvnn : int
        This number specifies the potential.
    lamb : float
        SRG \lambda parameter [fm^-1].
    integration_limits : tuple, optional
        Integration limits for a_2 calculation [fm^-1]. If no maximum is specified, then the function
        takes q to \infty.
    kmax : float, optional
        Maximum value in the momentum mesh [fm^-1].
    kmid : float, optional
        Mid-point value in the momentum mesh [fm^-1].
    ntot : int, optional
        Number of momentum points in mesh.
    edf : str, optional
        Name of EDF (e.g., 'SLY4').
    save_to_dropbox : bool, optional
        Option to save file to Dropbox directory.
    
    """

    # Make file name
    file_name_1 = 'a2_scaling_factors_nuclei' # First part of name
    # Add each nucleus to file name
    for nucleus in nuclei:
        file_name_1 += '_%s' % nucleus[0]
    # Use this second part to load the snmd files easily which are common
    file_name_2 = '_channels'
    # Add each channel to file name
    for channel in channels:
        file_name_2 += '_%s' % channel
    file_name_2 += '_kvnn_%d_lamb_%.2f_kmax_%.1f' % (kvnn, lamb, kmax)
    # Add integration limits to file name
    if len(integration_limits) == 1: # Assumes only minimum specified
        file_name = file_name_1 + file_name_2 + '_limits_%.1f_%.1f' % (integration_limits[0], kmax)
    elif len(integration_limits) == 2:
        file_name = file_name_1 + file_name_2 + '_limits_%.1f_%.1f' % (integration_limits[0],
                                                                       integration_limits[1])

    # Replace periods
    file_name = ff.replace_periods(file_name) + '.dat'
    
    # Create momentum mesh for integration
    if len(integration_limits) == 1:
        q_min = integration_limits[0]
        q_max = kmax # Corresponds to \infinity
        ntot_q = 80
    else:
        q_min = integration_limits[0]
        q_max = integration_limits[1]
        ntot_q = 40
    q_array, q_weights = gaussian_quadrature_mesh(q_max, ntot_q, xmin=q_min)

    # Initialize deuteron momentum distributions class for given potential
    dmd = deuteron_momentum_distributions(kvnn, lamb, kmax, kmid, ntot, interp=True)
    
    # Get interpolated functions of deuteron momentum distribution
    # Ignore the 1, \delta U, and \delta U^2 isolated contributions (take total only)
    n_d_func, _, _, _ = dmd.n_lambda_interp()
    
    # Calculate deuteron momentum distribution
    n_d_array = n_d_func(q_array)
    
    # Calculate deuteron denominator
    denominator = np.sum(q_weights*q_array**2*n_d_array)
    
    # Open file and write header where we allocate roughly 18 centered spaces for each label
    f = open(data_directory + '/%s/' % edf + file_name, 'w')
    header = '#' + '{:^17s}{:^18s}'.format('A', 'a_2')
    f.write(header + '\n') 

    # Loop over nuclei
    for nucleus in nuclei:

        # Details of nucleus
        nucleus_name = nucleus[0]
        Z = nucleus[1]
        N = nucleus[2]
        A = N + Z

        # Initialize single-nucleon momentum distributions class for given potential
        snmd = single_nucleon_momentum_distributions(kvnn, channels, lamb, kmax, kmid, ntot, interp=True)

        # Get interpolated functions of proton and neutron momentum distributions
        # Ignore the 1, \delta U, and \delta U^2 isolated contributions (take total only)
        n_p_func, _, _, _ = snmd.n_lambda_interp(nucleus_name, 'proton', Z, N, edf)
        n_n_func, _, _, _ = snmd.n_lambda_interp(nucleus_name, 'neutron', Z, N, edf)
    
        # Calculate each distribution for momenta q
        n_p_array = n_p_func(q_array)
        n_n_array = n_n_func(q_array)
        
        # Calculate probability distribution
        p_a_array = q_array**2/A * (n_p_array + n_n_array)
        
        # Calculate numerator
        numerator = np.sum(q_weights*p_a_array)
        
        # Compute a_2
        a_2 = numerator / denominator

        # Write to data file following the format from the header
        line = '{:^18n}{:^18.6f}'.format(A, a_2)
        f.write('\n' + line)

    # Close file
    f.close()

    # Save to Dropbox?
    if save_to_dropbox:
        _ = shutil.copy(data_directory + '/' + file_name, dropbox_directory)

In [25]:
# # a_2 scaling factors (integration from 2 to \infty fm^-1 and SLy4)

# integration_limits = (2.0,)

# t0 = time.time() # Start time
# a2_scaling_factors(nuclei_sly4, channels, kvnn, lamb, integration_limits, kmax, kmid, ntot, 'SLY4', save_to_dropbox)
# t1 = time.time() # Stop time
# mins = round( (t1 - t0) / 60.0, 5) # Minutes elapsed
# print('%.5f minutes elasped' % mins)

In [26]:
# # a_2 scaling factors (integration from 3.8 to 4.5 fm^-1 and SLy4)

# integration_limits = (3.8, 4.5)

# t0 = time.time() # Start time
# a2_scaling_factors(nuclei_sly4, channels, kvnn, lamb, integration_limits, kmax, kmid, ntot, 'SLY4', save_to_dropbox)
# t1 = time.time() # Stop time
# mins = round( (t1 - t0) / 60.0, 5) # Minutes elapsed
# print('%.5f minutes elasped' % mins)

In [27]:
# # a_2 scaling factors with \lambda=1.5, 2, and 6 fm^-1 (doing both integration limits from before and SLy4)

# lambdas = (1.5, 2.0, 6.0)
# integration_limits_tuple = ( (2.0,), (3.8, 4.5) )

# t0 = time.time() # Start time
# for ilamb in lambdas:
#     for iintegration_limits in integration_limits_tuple:
#         a2_scaling_factors(nuclei_sly4, channels, kvnn, ilamb, iintegration_limits, kmax, kmid, ntot, 'SLY4',
#                            save_to_dropbox)
# t1 = time.time() # Stop time
# mins = round( (t1 - t0) / 60.0, 5) # Minutes elapsed
# print('%.5f minutes elasped' % mins)

In [28]:
# # a_2 scaling factors (integration from 2 to \infty fm^-1 and Gogny)

# integration_limits = (2.0,)

# t0 = time.time() # Start time
# a2_scaling_factors(nuclei_gogny, channels, kvnn, lamb, integration_limits, kmax, kmid, ntot, 'Gogny', save_to_dropbox)
# t1 = time.time() # Stop time
# mins = round( (t1 - t0) / 60.0, 5) # Minutes elapsed
# print('%.5f minutes elasped' % mins)

In [29]:
# # a_2 scaling factors (integration from 3.8 to 4.5 fm^-1 and Gogny)

# integration_limits = (3.8, 4.5)

# t0 = time.time() # Start time
# a2_scaling_factors(nuclei_gogny, channels, kvnn, lamb, integration_limits, kmax, kmid, ntot, 'Gogny', save_to_dropbox)
# t1 = time.time() # Stop time
# mins = round( (t1 - t0) / 60.0, 5) # Minutes elapsed
# print('%.5f minutes elasped' % mins)

__Extras__

In [30]:
def deuteron_partial_wave_ratios(kvnn, lamb, kmax=15.0, kmid=3.0, ntot=120, save_to_dropbox=False):
    """
    3S1-only / 3S1-3D1 ratio of the \delta U^2 SRG-evolved deuteron pair momentum distribution term
    using deuteron wave function (not LDA).
    
    Parameters
    ----------
    kvnn : int
        This number specifies the potential.
    lamb : float
        SRG \lambda parameter [fm^-1].
    kmax : float, optional
        Maximum value in the momentum mesh [fm^-1].
    kmid : float, optional
        Mid-point value in the momentum mesh [fm^-1].
    ntot : int, optional
        Number of momentum points in mesh.
    save_to_dropbox : bool, optional
        Option to save file to Dropbox directory.

    """

    # Load momentum
    q_array, _ = vnn.load_momentum(kvnn, '3S1', kmax, kmid, ntot)
    
    # Initialize deuteron momentum distributions class for given potential
    dmd = deuteron_momentum_distributions(kvnn, lamb, kmax, kmid, ntot, interp=False)
    
    # Initialize file
    file_name = 'deuteron_partial_wave_ratios_kvnn_%d_lamb_%.2f_kmax_%.1f' % (kvnn, lamb, kmax)
    file_name = ff.replace_periods(file_name) + '.dat'
    f = open(data_directory + '/' + file_name, 'w')
    
    # Write header where we allocate roughly 18 centered spaces for each label
    header = '#' + '{:^17s}{:^18s}'.format('q', 'ratio')
    f.write(header + '\n')
    
    # Loop over momenta q
    for iq, q in enumerate(q_array):
        
        # Calculate ratio (don't need total)
        _, ratio = dmd.n_lambda_exact(q, contributions='partial_wave_ratio')

        # Write to data file following the format from the header
        line = '{:^18.6f}{:^18.6e}'.format(q, ratio)
        f.write('\n' + line)

    # Close file
    f.close()
    
    # Save to Dropbox?
    if save_to_dropbox:
        _ = shutil.copy(data_directory + '/' + file_name, dropbox_directory)

In [31]:
# # Deuteron pair momentum distribution 3S1 / 3S1-3D1 ratios

# t0 = time.time() # Start time
# deuteron_partial_wave_ratios(kvnn, lamb, kmax, kmid, ntot, save_to_dropbox)
# t1 = time.time() # Stop time
# mins = round( (t1 - t0) / 60.0, 5) # Minutes elapsed
# print('%.5f minutes elasped' % mins)

In [32]:
def factorization_ratios(k_0, k_points, kvnn, channel, lamb, kmax=15.0, kmid=3.0, ntot=120,
                         save_to_dropbox=False):
    """
    Ratio of \delta U \delta U^{\dagger} with respect to momentum [fm^-1] for k = k_points in the numerator
    and k = k_0 in the denominator, and k' = q_array.

    Parameters
    ----------
    k_0 : float
        Momentum value for U(k_0, q) in the denominator [fm^-1].
    k_points : 1-D ndarray
        Array of momentum values for U(k_points, q) in the numerator [fm^-1].
    kvnn : int
        This number specifies the potential.
    channel : str
        The partial wave channel (e.g. '1S0').
    lamb : float
        SRG \lambda parameter [fm^-1].
    kmax : float, optional
        Maximum value in the momentum mesh [fm^-1].
    kmid : float, optional
        Mid-point value in the momentum mesh [fm^-1].
    ntot : int, optional
        Number of momentum points in mesh.
    save_to_dropbox : bool, optional
        Option to save file to Dropbox directory.

    """
    
    # Make file name
    file_name = 'factorization_ratios_k_0_%s_k_points' % ff.convert_number_to_string(k_0)
    # Add each k_i to file name
    for k_i in k_points:
        file_name += '_%s' % ff.convert_number_to_string(k_i)
    file_name += '_kvnn_%d_%s_lamb_%.2f_kmax_%.1f' % (kvnn, channel, lamb, kmax)
    file_name = ff.replace_periods(file_name) + '.dat'
    
    # Load momentum and weights
    q_array, q_weights = vnn.load_momentum(kvnn, channel, kmax, kmid, ntot)
    # Use factor_array to divide out momenta/weights for mesh-independent result
    if vnn.coupled_channel(channel):
        factor_array = np.concatenate( (np.sqrt(q_weights) * q_array, np.sqrt(q_weights) * q_array) ) * \
                       np.sqrt(2/np.pi)
    else:
        factor_array = np.sqrt(2*q_weights/np.pi) * q_array
    col, row = np.meshgrid(factor_array, factor_array)
    
    # Load initial and evolved Hamiltonians
    H_initial = vnn.load_hamiltonian(kvnn, channel, kmax, kmid, ntot)
    H_evolved = vnn.load_hamiltonian(kvnn, channel, kmax, kmid, ntot, method='srg', lamb=lamb)
    
    # Calculate SRG transformation
    U_matrix_unitless = SRG_unitary_transformation(H_initial, H_evolved)
    
    # Converting to units [fm^3]
    I_matrix_unitless = np.eye( len(factor_array), len(factor_array) )
    delta_U_matrix_unitless = U_matrix_unitless - I_matrix_unitless
    delta_U_matrix = delta_U_matrix_unitless / row / col
    
    # Calculate \delta U(k_0, q) \delta U^{\dagger}(q, k_0) array
    k_0_index = op.find_q_index(k_0, q_array)
    # Single channel (do not include coupled parts for coupled-channel)
    denominator_array = delta_U_matrix[k_0_index, :ntot] * delta_U_matrix.T[:ntot, k_0_index]
    
    # Store each ratio in dictionary
    d = {}
    
    # Loop over k_i in k_points
    for k_i in k_points:
        
        # Index of k_i in q_array
        k_i_index = op.find_q_index(k_i, q_array)
            
        # Obtain U(k_i, q) array
        numerator_array = delta_U_matrix[k_i_index, :ntot] * delta_U_matrix.T[:ntot, k_i_index]
        
        # Calculate ratio and store in dictionary with k_i as the key
        d[k_i] = abs(numerator_array / denominator_array)

    # Open file and write header where we allocate roughly 18 centered spaces for each label
    f = open(data_directory + '/' + file_name, 'w')
    
    # Header should show each momentum q and each k_i as column label
    header = '#' + '{:^17s}'.format('q')
    
    # Loop over k_i
    for k_i in k_points:
        
        # Add each k_i as column label
        header += '{:^18s}'.format( ff.convert_number_to_string(k_i) )

    f.write(header + '\n')
    
    # Loop over momenta q
    for iq, q in enumerate(q_array):
        
        line = '{:^18.6f}'.format(q)
        
        # Loop over each k_i and write each ratio
        for k_i in k_points:
            
            # Get ratio for given q and k_i
            ratio = d[k_i][iq] # k_i is a key and iq is an index here
            line += '{:^18.6e}'.format(ratio)
            
        f.write('\n' + line)

    # Close file
    f.close()
    
    # Save to Dropbox?
    if save_to_dropbox:
        _ = shutil.copy(data_directory + '/' + file_name, dropbox_directory)

In [33]:
# # Factorization ratios \delta U(k_i, q) \delta U^\dagger(q, k_i) / \delta U(k_0, q) \delta U^\dagger(q, k_0)

# # Do several k_0 values
# k_0_values = np.array([0.1, 0.2, 0.3, 0.4])
# k_points = np.array([0.5, 1.0, 1.5, 3.0])
# channel = '3S1'

# t0 = time.time() # Start time
# for k_0 in k_0_values:
#     factorization_ratios(k_0, k_points, kvnn, channel, lamb, kmax, kmid, ntot, save_to_dropbox)
# t1 = time.time() # Stop time
# mins = round( (t1 - t0) / 60.0, 5) # Minutes elapsed
# print('%.5f minutes elasped' % mins)

In [34]:
def factorization_ratios_k_dependence(q_0, q_points, kvnn, channel, lamb, kmax=15.0, kmid=3.0, ntot=120,
                                       save_to_dropbox=False):
    """
    Ratio of \delta U \delta U^{\dagger} with respect to momentum [fm^-1] for q = q_points in the numerator
    and q = q_0 in the denominator, and k = k_array. This is very similar to factorization_ratios but shows
    the dependence on low-k instead.

    Parameters
    ----------
    q_0 : float
        Momentum value for U(k, q_0) in the denominator [fm^-1].
    q_points : 1-D ndarray
        Array of momentum values for U(k, q_points) in the numerator [fm^-1].
    kvnn : int
        This number specifies the potential.
    channel : str
        The partial wave channel (e.g. '1S0').
    lamb : float
        SRG \lambda parameter [fm^-1].
    kmax : float, optional
        Maximum value in the momentum mesh [fm^-1].
    kmid : float, optional
        Mid-point value in the momentum mesh [fm^-1].
    ntot : int, optional
        Number of momentum points in mesh.
    save_to_dropbox : bool, optional
        Option to save file to Dropbox directory.

    """
    
    # Make file name
    file_name = 'factorization_ratios_q_0_%s_q_points' % ff.convert_number_to_string(q_0)
    # Add each k_i to file name
    for q_i in q_points:
        file_name += '_%s' % ff.convert_number_to_string(q_i)
    file_name += '_kvnn_%d_%s_lamb_%.2f_kmax_%.1f' % (kvnn, channel, lamb, kmax)
    file_name = ff.replace_periods(file_name) + '.dat'
    
    # Load momentum and weights
    k_array, k_weights = vnn.load_momentum(kvnn, channel, kmax, kmid, ntot)
    # Use factor_array to divide out momenta/weights for mesh-independent result
    if vnn.coupled_channel(channel):
        factor_array = np.concatenate( (np.sqrt(k_weights) * k_array, np.sqrt(k_weights) * k_array) ) * \
                       np.sqrt(2/np.pi)
    else:
        factor_array = np.sqrt(2*k_weights/np.pi) * k_array
    col, row = np.meshgrid(factor_array, factor_array)
    
    # Load initial and evolved Hamiltonians
    H_initial = vnn.load_hamiltonian(kvnn, channel, kmax, kmid, ntot)
    H_evolved = vnn.load_hamiltonian(kvnn, channel, kmax, kmid, ntot, method='srg', lamb=lamb)
    
    # Calculate SRG transformation
    U_matrix_unitless = SRG_unitary_transformation(H_initial, H_evolved)
    
    # Converting to units [fm^3]
    I_matrix_unitless = np.eye( len(factor_array), len(factor_array) )
    delta_U_matrix_unitless = U_matrix_unitless - I_matrix_unitless
    delta_U_matrix = delta_U_matrix_unitless / row / col
    
    # Calculate \delta U(k, q_0) \delta U^{\dagger}(q_0, k) array
    q_0_index = op.find_q_index(q_0, k_array)
    # Single channel (do not include coupled parts for coupled-channel)
    denominator_array = delta_U_matrix[:ntot, q_0_index] * delta_U_matrix.T[q_0_index, :ntot]
    
    # Store each ratio in dictionary
    d = {}
    
    # Loop over q_i in q_points
    for q_i in q_points:
        
        # Index of q_i in k_array
        q_i_index = op.find_q_index(q_i, k_array)
            
        # Obtain U(k_i, q) array
        numerator_array = delta_U_matrix[:ntot, q_i_index] * delta_U_matrix.T[q_i_index, :ntot]
        
        # Calculate ratio and store in dictionary with k_i as the key
        d[q_i] = abs(numerator_array / denominator_array)

    # Open file and write header where we allocate roughly 18 centered spaces for each label
    f = open(data_directory + '/' + file_name, 'w')
    
    # Header should show each momentum k and each q_i as column label
    header = '#' + '{:^18s}'.format('k')
    
    # Loop over k_i
    for q_i in q_points:
        
        # Add each q_i as column label
        header += '{:^18s}'.format( ff.convert_number_to_string(q_i) )

    f.write(header + '\n')
    
    # Loop over momenta q
    for ik, k in enumerate(k_array):
        
        line = '{:^18.6f}'.format(k)
        
        # Loop over each q_i and write each ratio
        for q_i in q_points:
            
            # Get ratio for given k and q_i
            ratio = d[q_i][ik] # q_i is a key and ik is an index here
            line += '{:^18.6e}'.format(ratio)
            
        f.write('\n' + line)

    # Close file
    f.close()
    
    # Save to Dropbox?
    if save_to_dropbox:
        _ = shutil.copy(data_directory + '/' + file_name, dropbox_directory)

In [35]:
# # Factorization ratios \delta U(k, q_i) \delta U^\dagger(q_i, k) / \delta U(k, q_0) \delta U^\dagger(q_0, k)

# # Do q_0 = 6.0 fm^-1
# q_0 = 6.0
# q_points = np.array([1.0, 2.0, 3.0, 4.0])
# channel = '3S1'

# t0 = time.time() # Start time
# factorization_ratios_k_dependence(q_0, q_points, kvnn, channel, lamb, kmax, kmid, ntot, save_to_dropbox)
# t1 = time.time() # Stop time
# mins = round( (t1 - t0) / 60.0, 5) # Minutes elapsed
# print('%.5f minutes elasped' % mins)

In [36]:
def absolute_contacts(nuclei, channels, kvnn, lamb, integration_limits=(2.0,), kmax=15.0, kmid=3.0, ntot=120,
                      edf='SLY4', save_to_dropbox=False):
    """
    Nuclear contacts evaluated using HF+LDA, SRG-evolved pair momentum distributions. The integration limits are
    set to 2-\infty fm^-1 by default.
    
    Parameters
    ----------
    nuclei : tuple
        Tuple that contains the details for various nuclei formatted as another tuple:
        (name (str), Z (int), N (int)) (e.g., nuclei[i] = ('O16', 8, 8)).
    channels : tuple
        Partial wave channels to include in the calculation (e.g., ('1S0', '3S1')).
    kvnn : int
        This number specifies the potential.
    lamb : float
        SRG \lambda parameter [fm^-1].
    integration_limits : tuple, optional
        Integration limits for calculation [fm^-1]. If no maximum is specified, then the function
        takes q to \infty.
    kmax : float, optional
        Maximum value in the momentum mesh [fm^-1].
    kmid : float, optional
        Mid-point value in the momentum mesh [fm^-1].
    ntot : int, optional
        Number of momentum points in mesh.
    edf : str, optional
        Name of EDF (e.g., 'SLY4').
    save_to_dropbox : bool, optional
        Option to save file to Dropbox directory.
    
    """

    # Make file name
    file_name_1 = 'absolute_contacts_nuclei' # First part of name
    # Add each nucleus to file name
    for nucleus in nuclei:
        file_name_1 += '_%s' % nucleus[0]
    # Use this second part to load the snmd files easily which are common
    file_name_2 = '_channels'
    # Add each channel to file name
    for channel in channels:
        file_name_2 += '_%s' % channel
    file_name_2 += '_kvnn_%d_lamb_%.2f_kmax_%.1f' % (kvnn, lamb, kmax)
    # Add integration limits to file name
    if len(integration_limits) == 1: # Assumes only minimum specified
        file_name = file_name_1 + file_name_2 + '_limits_%.1f_%.1f' % (integration_limits[0], kmax)
    elif len(integration_limits) == 2:
        file_name = file_name_1 + file_name_2 + '_limits_%.1f_%.1f' % (integration_limits[0],
                                                                       integration_limits[1])

    # Replace periods
    file_name = ff.replace_periods(file_name) + '.dat'
    
    # Create momentum mesh for integration
    if len(integration_limits) == 1:
        q_min = integration_limits[0]
        q_max = kmax # Corresponds to \infinity
        ntot_q = 80
    else:
        q_min = integration_limits[0]
        q_max = integration_limits[1]
        ntot_q = 40
    # q_array, q_weights = gaussian_quadrature_mesh(q_max, ntot_q, xmin=q_min)
    q_array, q_weights = vnn.load_momentum(kvnn, '3S1', kmax, kmid, ntot)
    
#     2 * (2/np.pi)**2 # DMD
#     4*np.pi * (2/np.pi)**2 * (2*np.pi)**3/(4*np.pi) # PMD
#     4*np.pi * 2 * (2/np.pi)**2 # SNMD

    # Open file and write header where we allocate roughly 18 centered spaces for each label
    f = open(data_directory + '/%s/' % edf + file_name, 'w')
    header = '#' + '{:^17s}{:^18s}{:^18s}{:^18s}'.format('A', 'C^{S=1}_{pn}', 'C^{S=0}_{pn}', 'C^{S=0}_{pp}')
    f.write(header + '\n')
    
    # Initialize deuteron momentum distributions class for given potential
    dmd = deuteron_momentum_distributions(kvnn, lamb, kmax, kmid, ntot, interp=True)
    
    # Get interpolated functions of deuteron momentum distribution
    # Ignore the 1, \delta U, and \delta U^2 isolated contributions (take total only)
    n_d_func, _, _, _ = dmd.n_lambda_interp()
    
    # Calculate deuteron momentum distribution
    n_d_array = n_d_func(q_array)
    
    # Deuteron C^{S=1}_{pn}
    # A/2 factor is 1
    c_d = 4*np.pi/(2*np.pi)**3 * np.sum(q_weights*q_array**2*n_d_array) * 100
    print(c_d)
    
    # Write deuteron to file (last two columns are blank since deuteron is pn and 3S1 only)
    line = '{:^18n}{:^18.6f}'.format(2, c_d)
    f.write('\n' + line)
    
    # For A > 2, calculate pair momentum distributions at Q = 0 for 1S0 and 3S1 seperately
    pmd_1S0 = pair_momentum_distributions(kvnn, ['1S0'], lamb, kmax, kmid, ntot, interp=False)
    pmd_3S1 = pair_momentum_distributions(kvnn, ['3S1'], lamb, kmax, kmid, ntot, interp=False)  

    # Loop over nuclei
    for i, nucleus in enumerate(nuclei):

        # Details of nucleus
        nucleus_name = nucleus[0]
        Z = nucleus[1]
        N = nucleus[2]
        A = N + Z
        
        # Divide contacts by A/2 and multiply by 100
        factor =  100 * 2 / A
        
        # Load R values and nucleonic densities (the R_array's are the same)
        R_array, rho_p_array = load_density(nucleus_name, 'proton', Z, N, edf)
        R_array, rho_n_array = load_density(nucleus_name, 'neutron', Z, N, edf)
        dR = R_array[2] - R_array[1] # Step-size
        
        # pn and S = 1
        n_pn_3S1_array = 2*pmd_3S1.n_Q0(q_array, R_array, dR, rho_p_array, rho_n_array)
        c_pn_3S1 = factor * 4*np.pi/(2*np.pi)**3 * np.sum( q_weights * q_array**2 * n_pn_3S1_array )
        
        # pn and S = 0
        n_pn_1S0_array = 2*pmd_1S0.n_Q0(q_array, R_array, dR, rho_p_array, rho_n_array)
        # print( 4*np.pi/(2*np.pi)**6 * np.sum( q_weights * q_array**2 * (n_pn_3S1_array+n_pn_1S0_array) ) * 2/(N*Z) )
        c_pn_1S0 = factor * 4*np.pi/(2*np.pi)**3 * np.sum( q_weights * q_array**2 * n_pn_1S0_array )
        
        # pp and S = 0
        n_pp_1S0_array = pmd_1S0.n_Q0(q_array, R_array, dR, rho_p_array)
        c_pp_1S0 = factor * 4*np.pi/(2*np.pi)**3 * np.sum( q_weights * q_array**2 * n_pp_1S0_array )

        # Write to data file following the format from the header
        line = '{:^18n}{:^18.6f}{:^18.6f}{:^18.6f}'.format(A, c_pn_3S1, c_pn_1S0, c_pp_1S0)
        f.write('\n' + line)

    # Close file
    f.close()

    # Save to Dropbox?
    if save_to_dropbox:
        _ = shutil.copy(data_directory + '/' + file_name, dropbox_directory)

# Should be pretty similar to a2 calculation
# With nuclei_sly4 do C^{S=1}_{pn}, C^{S=0}_{pn}, and C^{S=0}_{pp}
# Divide by A/2 and multiply by 100?
# Alvioli paper says: C^{S=1}_{pn} =  lim k > 1.5 fm^-1 n^A(k, K=0)/n^d(k)
# Weiss
# Try same as a2 calculation but set Q=0 (q from 2 to \infty fm^-1)
# If that doesn't look right, integrate out Q dependence

In [37]:
# # Absolute contacts (integration from 2 to \infty fm^-1 and SLy4)

# integration_limits = (2.0,)

# t0 = time.time() # Start time
# absolute_contacts(nuclei_sly4, channels, kvnn, lamb, integration_limits, kmax, kmid, ntot, 'SLY4', save_to_dropbox)
# t1 = time.time() # Stop time
# mins = round( (t1 - t0) / 60.0, 5) # Minutes elapsed
# print('%.5f minutes elasped' % mins)

99.81667195596418
1.2479629289171565
0.7913612562775648
0.7556268391833796
0.6078802212610791
0.5616469032084203
0.5421357552756657
0.4665578258747081
0.19964 minutes elasped


In [38]:
# # Absolute contacts (integration from 3.8 to 4.5 fm^-1 and SLy4)

# integration_limits = (3.8, 4.5)

# t0 = time.time() # Start time
# absolute_contacts(nuclei_sly4, channels, kvnn, lamb, integration_limits, kmax, kmid, ntot, 'SLY4', save_to_dropbox)
# t1 = time.time() # Stop time
# mins = round( (t1 - t0) / 60.0, 5) # Minutes elapsed
# print('%.5f minutes elasped' % mins)

99.81667195596418
1.2479629289171565
0.7913612562775648
0.7556268391833796
0.6078802212610791
0.5616469032084203
0.5421357552756657
0.4665578258747081
0.19886 minutes elasped
